In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import torch
import os
from components import rag
from openai import OpenAI


In [ ]:
# Example usage of RAG:
query = "What is the main goal of the project launched in Cambodia on February 27 to improve plastic waste management?"
document_dataset = "./data/1K_news.csv"
output = rag.query_answering_system(query, document_dataset)
print('Answer: {}'.format(output['answer']))
print('Reference1: {}'.format(output['title'][0]))
print('Reference1: {}'.format(output['title'][1]))
print('Reference1: {}'.format(output['title'][2]))

In [ ]:
# Generate questions by Chatgpt
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

df_1K = pd.read_csv('./data/1K_news.csv', encoding='utf-8')


# Ensure df exists and contains the necessary columns
df = df_1K.head(100)  # Select the first 50 rows

def generate_question(text):
    prompt = (
        "Generate a simple, easy-to-understand and not too long question based on the following news content."
        "Also, the question must be specific and clear. For example, instead of using 'servants,' it should specify which region or country’s servants are being referred to."
        "Additionally, the question should not be too difficult, and the answer must be explicitly contained within the following News content.\n\n"
        f"News content:\n{text}\n\n"
        "Question:"
    )

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content.strip()

# Apply the function to generate questions
df["generated_question"] = df["text"].apply(generate_question)

# Save the results to a CSV file
df.to_csv("./data/Q6/100_news_questions.csv", index=False, encoding='utf-8')

print("Question generation completed. Results saved to news_questions.csv")


In [ ]:
# Generate answers by Rag
df = pd.read_csv('./data/Q6/100_news_questions.csv', encoding='utf-8')
# df = df.head(5)
document_dataset = "./data/1K_news.csv"
for i in range(len(df)):
    query = df.loc[i, 'generated_question']
    output = rag.query_answering_system(query, document_dataset)
    query = df.loc[i, 'generated_response'] = output['answer']
    query = df.loc[i, 'retrieved_docs_id'] = str(output['retrieved_docs_id'])[1:-1]

    print(i)

df.to_csv("./data/Q6/100_news_QA.csv", index=False, encoding='utf-8')

In [ ]:
# Precision@K
df = pd.read_csv('./data/Q6/100_news_QA.csv', encoding='utf-8')
TP_3 = 0
TP_2 = 0
TP_1 = 0
for i in range(len(df)):

    content_id = str(df['content_id'][i])
    retrieved_docs_id = df['retrieved_docs_id'][i].split(', ')
    # print(content_id)
    # print(retrieved_docs_id)

    if content_id in retrieved_docs_id:
        TP_3 += 1
        TP_2 += 1
        TP_1 += 1
    elif content_id in retrieved_docs_id[:-1]:
        TP_2 += 1
        TP_1 += 1
    elif content_id in retrieved_docs_id[:-2]:
        TP_1 += 1

Precision_top3 = TP_3 / len(df)
Precision_top2 = TP_2 / len(df)
Precision_top1 = TP_1 / len(df)

print(f"Precision@3: {Precision_top3:.4f}")
print(f"Precision@2: {Precision_top2:.4f}")
print(f"Precision@1: {Precision_top1:.4f}")

In [ ]:
# Use Chat GPT evaluate if Rag has model hallucinated
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

def evaluate_rag_hallucinated(text, question, answer):
    prompt = (
        "Evaluate whether the Knowledge-based AI assistant's answer is factually accurate based only on the given documents.\n"
        "Your response should be in two rows:\n"
        "1. First row: True or False (True if the answer is fully supported by the documents, False if it contains hallucinated or unverifiable information).\n"
        "2. Second row: A short explanation for your decision.\n\n"
        "Documents:\n"
        f"{text}\n\n"
        "Question:\n"
        f"{question}\n\n"
        "Answer from the AI assistant:\n"
        f"{answer}\n\n"
    )


    # print(prompt)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content.strip()

df = pd.read_csv('./data/Q6/100_news_QA.csv', encoding='utf-8')
df_1K = pd.read_csv('./data/1K_news.csv', encoding='utf-8')

df["hallucinated"] = None
df["reason"] = None

for i in range(len(df)):

    doc_ids = list(map(int, df["retrieved_docs_id"][i].split(", ")))
    text_docs = df_1K[df_1K['content_id'].isin(doc_ids)]['text']
    text = "\n\n".join([f"document{j+1}: {text_docs.iloc[j]}" for j in range(min(3, len(text_docs)))])
        
    isHallucinated, reason = evaluate_rag_hallucinated(text, df.loc[i, "generated_question"], df.loc[i, "generated_response"]).splitlines()
    df.loc[i, 'hallucinated'] = isHallucinated
    df.loc[i, 'reason'] = reason

print(df['hallucinated'])
df.to_csv("./data/Q6/100_news_QA.csv", index=False, encoding='utf-8')



In [ ]:
list(df['hallucinated'])
